#### Задание 2. Проверить, насколько хорошо работает NER
Данные брать из Index of /pub/named_entities

* проверить NER из nltk/spacy/deeppavlov.
* написать свой NER, попробовать разные подходы:
    * передаём в сетку токен и его соседей.
    * передаём в сетку только токен.
    * свой вариант.
* сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [ ]:
# загрузка датасета

%cd dataset
!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip     

In [25]:
import pandas as pd
import numpy as np
import corus

In [5]:
from corus import load_ne5

dir = 'dataset/Collection5/'
records = load_ne5(dir)
next(records)


Ne5Markup(
    id='1047',
    text='Жириновский предлагает обменять с США Сноудена на Бута\r\n\r\nЛидер ЛДПР Владимир Жириновский предложил обменять бывшего сотрудника ЦРУ США Эдварда Сноудена, который прибыл в Москву, на осужденного в Америке бизнесмена Виктора Бута.\r\n\r\n"Сноудена ни в коем случае не высылать в США, а обменять на Виктора Бута и Константина Ярошенко. В идеале — добавить генерала Олега Калугина", — написал он в своем микроблоге в Twitter.\r\n\r\nСноуден, работавший на компанию Booz Allen Hamilton — подрядчика Центрального разведывательного управления США, в начале июня распространил секретный ордер суда, по которому спецслужбы получили доступ ко всем звонкам крупнейшего сотового оператора Verizon, а также данные о сверхсекретной программе агентства национальной безопасности PRISM, позволяющей отслеживать электронные коммуникации на крупнейших сайтах. В воскресенье стало известно, что Сноуден прибыл из Гонконга в Москву и запросил убежища в Эквадоре.\r\n\r\nЧто ждет Э

In [6]:
from razdel import tokenize

In [7]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [8]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [9]:
df_words['tag'].value_counts()

OUT         219014
PER          21178
ORG          13641
LOC           4564
GEOPOLIT      4349
MEDIA         2481
Name: tag, dtype: int64

In [10]:
df_words.head(3)

,word,tag
0,Д,PER
1,.,PER
2,Медведев,PER


In [11]:
df_words.shape

(265227, 2)

### NLTK

In [19]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/alexey/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [21]:
nltk.pos_tag(df_words.word)[:10]

[('Д', 'NN'),
 ('.', '.'),
 ('Медведев', 'CC'),
 ('назначил', 'JJ'),
 ('ряд', 'NNP'),
 ('глав', 'NNP'),
 ('региональных', 'NNP'),
 ('МВД', 'NNP'),
 ('Президент', 'NNP'),
 ('России', 'NNP')]

In [25]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/alexey/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/alexey/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [34]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(
df_words.word[:500])) if hasattr(chunk, 'label') }

{('Андрей Головизнин', 'PERSON'),
 ('Андрей Конев', 'PERSON'),
 ('Андрей Ларионов', 'PERSON'),
 ('Болгов', 'PERSON'),
 ('Василий Романиц', 'PERSON'),
 ('Виктор Кошелев', 'PERSON'),
 ('Виталия Шевченко', 'PERSON'),
 ('Владимир Романюк', 'ORGANIZATION'),
 ('Владимира Овечкина', 'PERSON'),
 ('Кошкин', 'PERSON'),
 ('Кроме', 'PERSON'),
 ('МВД', 'ORGANIZATION'),
 ('Михаил Закомалдин', 'PERSON'),
 ('Михаил Миндзаев', 'PERSON'),
 ('Михаил Молоканов', 'PERSON'),
 ('Михаил Черников', 'PERSON'),
 ('Напомним', 'PERSON'),
 ('Никитин', 'PERSON'),
 ('Николай Максимов', 'PERSON'),
 ('Николай Симаков', 'PERSON'),
 ('Николаю Овчинникову', 'PERSON'),
 ('Николая Руя', 'PERSON'),
 ('России Дмитрий Медведев', 'PERSON'),
 ('Сергей Глушков', 'PERSON'),
 ('Сергей Корнильцев', 'PERSON'),
 ('Сергей Семенов', 'PERSON'),
 ('Сергей Титов', 'PERSON'),
 ('Сергея Широкова', 'PERSON'),
 ('Торопин', 'PERSON'),
 ('УВД', 'ORGANIZATION'),
 ('Указом Д', 'PERSON'),
 ('Югре', 'PERSON'),
 ('Южному', 'PERSON'),
 ('Якутия', 'GPE

### SPACY

In [48]:
import spacy
from spacy import displacy
spacy.load('ru_core_news_sm')
import ru_core_news_sm

In [51]:
nlp = ru_core_news_sm.load()
article = nlp(' '.join(df_words.word[:1000]))
displacy.render(article, jupyter=True, style='ent')

### Deeppavlov

In [ ]:
import deeppavlov
from deeppavlov import configs, build_model

deeppavlov_ner = build_model(configs.ner.ner_rus_bert, download=False)
rus_document = ' '.join(df_words.word[:500])
deeppavlov_ner([rus_document])